In [75]:
import pandas, numpy as np,re
import xml, itertools, re, xml.etree.ElementTree as ET
from html.parser import HTMLParser
import datetime, tqdm
pars =HTMLParser()

def prettyPrintXML(s):
    r = xml.dom.minidom.parseString(s) #r.RegistroXML)
    print(r.toprettyxml())
    
#from datetime import datetime as dt
#import locale
#locale.setlocale(locale.LC_ALL, 'es_es')
#spanishDateToPandas = lambda s: pandas.to_datetime(dt.strptime(s, '%d %b %Y %H:%M:%S'))

# Data reading and basic cleaning

- Find candidates of repeated pateints
- Find the used codes for diagnosis and procedures

In [82]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('Venezolanas/procedimientos.csv', index_col = 0)
procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)

#str2Date = lambda s:datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f') if isinstance(s, str) \
#           else datetime.datetime(1, 1, 1)
registros.FechaInicioAtencion = registros.FechaInicioAtencion.map(pandas.to_datetime) #registros.FechaInicioAtencion.map(str2Date)
procedimientos.FechaDescripcion  = procedimientos.FechaDescripcion.map(pandas.to_datetime)
from html.parser import HTMLParser
pars =HTMLParser()
#pars.unescape

C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,27,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [77]:
procedureIDFound = set()
abortions = []
def getProcFromXMLString(s):
    s = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', s)
    return s[0]

for k, p in procedimientos.iterrows():
    et = xml.etree.ElementTree.fromstring(p.XmlDescripcion)
    s = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', p.XmlDescripcion)
    for ss in s:
        procedureIDFound.add(ss)
    if 'H3094' in p.XmlDescripcion:
        abortions.append(k)
procedureIDFound = list(sorted(procedureIDFound))

In [79]:
procedimientosDesc.loc[procedureIDFound].to_excel('procedimientosFiltered.xlsx')
procedimientosDesc.loc[procedureIDFound].to_csv('procedimientosFiltered.csv')

In [8]:
unique = registros.Diagnostico.unique()
unique = [u for u in unique if isinstance(u, str)]
for u in unique:
    if u.upper() not in diagnosis.index:
        print(u)

0800


In [9]:
#Check duplicates by birthData
def candidateToRepeated(p1, p2):
    """
    Selects as candidate if they were born in the same name:
    - they have the same 1st surname
    - [TODO] they have similar height (~2cm), as found in the cases
    
    TODO: beware the case of twins!
    """
    #First surname criterio
    if p1[1].Ape1Afil.strip().lower() == p2[1].Ape1Afil.strip().lower():
        return True
    #TODO: Search height and see it's similar
    if False:
        return True
    #Otherwise return False
    return False
    

#Beware
candidates = []
threshold = '2010'
for c in pacientes.groupby('FechaNac'):
    #Arbitrary threshold for separating between mothers and borns
    if len(c[1]) >= 2 and c[0][:4] < threshold:
        for p1, p2 in itertools.combinations(c[1].iterrows(), r = 2):
            if p1[0] != p2[0] and candidateToRepeated(p1, p2):
                candidates.append([p1[0], p2[0]])
pandas.DataFrame.from_records(candidates).to_csv('repeatedCandidates.csv')

# Select the cases by procedure

- Based on the procedure, select the history register associated.
- TODO: Check that diagnosis and procedure are compatible

In [10]:
registersByPatient = registros.groupby('NumeroHistoria')

In [11]:
####
## Old code, before I discovered the link between case and operations
####
#getprocID = lambda s:  re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', s)[0]
#getprocDate = lambda s:  re.findall('<fechaCirugia>(.*)</fechaCirugia>', s)[0]
#
#def associateCaseAndHistory(interventionId, intervention, registersByPatient, 
#                            nDaysThreshold = 3,  nDaysUpperThreshold = 0):
#    pId = intervention.IdPaciente
#    df = registersByPatient.get_group(pId)
#    #interventionDate = spanishDateToPandas(getprocDate(intervention.XmlDescripcion))
#    interventionDate = pandas.to_datetime(intervention.FechaRegistro.replace('T', ''))
#    df = df.loc[df.FechaInicioAtencion.between(interventionDate - pandas.Timedelta(nDaysThreshold, 'D'),
#                                      interventionDate +  pandas.Timedelta(nDaysUpperThreshold, 'D'))]
#    #Do not count Triage in the cases ID
#    caseIds =  [s for s in df.Caso.unique() if not s.startswith('TRI')]
#    if len(caseIds) == 0:
#        print('Error in intervention %d found %d. Code = %s. Patient = %s' % (interventionId, df.Caso.nunique(),
#                                                                getprocID(intervention.XmlDescripcion), intervention.IdPaciente ))
#        print(intervention.FechaDescripcion, intervention.FechaRegistro, interventionDate)
#        return False
#    else:
#        return df.loc[df.FechaInicioAtencion == df.FechaInicioAtencion.max()].Caso.values[0]
#Identify borns and abortions using the proc ID
#interventionToCase = {}
#for k, i in tqdm.tqdm_notebook(procedimientos.iterrows()):
#    interventionToCase[k] = (associateCaseAndHistory(k, i, registersByPatient), i.IdPaciente)


In [12]:
interventionToCase = {}
for i,r  in tqdm.tqdm_notebook(registros.iterrows()):
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcion = int(et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo'))
        interventionToCase[idDescripcion] = (r.Caso, r.NumeroHistoria, i)

In [13]:
errors = []
for i, (_, k, k2) in interventionToCase.items():
    if procedimientos.loc[i].IdPaciente != k:
        print('ERROR: Patient name does not coincides', i, k2)
        errors.append(i)
for i in errors:
    del interventionToCase[i]

ERROR: Patient name does not coincides 61066 1356698


# Parse information from cases
Now we need to parse the information from the cases

In [14]:
def getStateDataFromCase(case):
    """
    Get some information from the case
    """
    data = {}
    r = ET.fromstring(case.RegistroXML)
    try:
        data['FC']=r.find('.//row[@NombreCampo="FrecuenciaCardiaca"]').get('ValorCampo')
        data['FR']=r.find('.//row[@NombreCampo="FrecuenciaRespiratoria"]').get('ValorCampo')
        data['Presion']=r.find('.//row[@NombreCampo="Presion"]').get('ValorCampo')
        data['Temperatura']=r.find('.//row[@NombreCampo="Temperatura"]').get('ValorCampo')
        data['Talla']=r.find('.//row[@NombreCampo="Talla"]').get('ValorCampo')
        data['Peso']=r.find('.//row[@NombreCampo="Peso"]').get('ValorCampo')
        return data
    except:
        return {}

In [ ]:
G1 P2 C4, A4

In [15]:
floatParse = '[0-9]*[\.,]?[0-9]+'
compareGPCA = lambda s1, s2: all([i == j or not i or not j for i, j in zip(s1, s2)])
allEqual = lambda s: all([compareGPCA(i, j) for i, j in  itertools.combinations(s, r = 2)])
sep= '\s*[,;:]?\s*'
 #In some cases there is only G
patternGPAC = 'G%s(?P<G>[0-9]+)' %sep + sep + 'P%s(?P<P>[0-9]+)'  %sep + sep + '(?:'
patternGPAC += 'A%s(?P<A>[0-9]+)' %sep  + sep + 'C%s(?P<C>[0-9]+)'  %sep
patternGPAC += '|'
patternGPAC += 'C%s(?P<C2>[0-9]+)' %sep  + sep + 'A%s(?P<A2>[0-9]+)' %sep
patternGPAC += '|' # C and A can be missing, or in other order
patternGPAC += ')'
searchGPAC = re.compile(patternGPAC, flags=0)

meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
meses = meses + list(map(lambda s: s[:3], meses))
noRecuerda = ['NO', '\?']
separadorFecha = '(?:[\.\\/-]|DE|DEL|\s)'
date =  '\(?' +  '((?:[0-9]+)'+ sep + separadorFecha + sep + '(?:[0-9]+|%s)'%  '|'.join(meses) + \
                       sep + separadorFecha + sep + '(?:[0-9]+))' + '\)?' 

searchFUM = re.compile('FUM'+ sep + '(?::|.)?'+ sep +'(' +  date + '|%s)' % ('|'.join(noRecuerda)), flags = re.IGNORECASE)
errors = []
def parseGPCA_and_fum(s):
    """
    Gets the GPCA and FUM from the Triage or epicrisis
    
    TODO: actually, registro del recien nacido has it as a field.
    """
    r = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',ET.fromstring(s).findall('row')))
    if len(r) == 0:
         return {}
    text = pars.unescape(r[0].attrib['ValorCampo'])
    
    allowedStarts = ['7-&gt;', '-', '- ANTECEDENTES']
    line = re.findall('^%s(?:%s)?%s' % (sep, '|'.join(allowedStarts), sep) + 'G' + sep + '[0-9]+.*$', text, re.M)
    if len(line) != 1:
        #If it doesn't find it, search brutally
        search = searchGPAC.findall(s)
        searchCleaned = [(s[0], s[1], s[2] + s[-1], s[3] + s[-2]) for s in search]
        if search and allEqual(search):
            GPCA_OK = True
            parsedGPCA = search[0]
        else:
            GPCA_OK = False
            parsedGPCA = []

    else:
        f = lambda s: s[0] if s else ''
        G = f(re.findall('G' + sep + '([0-9]+)', line[0]))
        A = f(re.findall('A' + sep + '([0-9]+)', line[0]))
        C = f(re.findall('C' + sep + '([0-9]+)', line[0]))
        P = f(re.findall('P' + sep + '([0-9]+)', line[0])) 
        parsedGPCA = [G, P, A, C]
        GPCA_OK = True
    #Prob athere is a better way...
    parsedFUM = searchFUM.findall(s)
    return {'fum' : len(parsedFUM) > 0,
            'fum_OK' : len(parsedFUM) ,
            'GPCA_OK' : GPCA_OK,
            'fum_Data' : parsedFUM,
            'parsedGPCA' : parsedGPCA}

In [16]:
from IPython.display import display

def getBackgroundRegister(casoID, registrosByCase):
    """
    Gets a register that includes the personal background in order Epicrisis / Triage / Ingreso a urgencias
    """
    #
    epicrisis = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Epicrisis', case = False) == True)]
    if epicrisis is not None and len(epicrisis):
        return epicrisis
    
    triage = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Triage', case = False) == True)]
    if triage is not None and len(triage):
        return triage

    urgencias = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Ingreso de Urgencias', case = False) == True)]
    if urgencias is not None:
        return urgencias


In [17]:
errorsBackgroundRegister = []
backgroundByCase = {}
dataGPAC_fum = {}
for k, (case, pId, _) in tqdm.tqdm_notebook(interventionToCase.items()):
    if case is False:
        continue
        
    backgroundByCase[case] = getBackgroundRegister(case, registersByPatient.get_group(pId))
    if len(backgroundByCase[case]) == 0:
        errorsBackgroundRegister.append(case)
    else: 
        dataGPAC_fum[case] = parseGPCA_and_fum(backgroundByCase[case].iloc[0].RegistroXML)
        c = getProcFromXMLString(procedimientos.loc[k].XmlDescripcion)
        #dataGPAC_fum[case] ['ProcCode'] = c
        #dataGPAC_fum[case] ['ProcDesc'] = procedimientosDesc.loc[c].DESCRIP
    
print('No background', len(errorsBackgroundRegister))

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.



No background 5


In [18]:
backgroundByCase[case]

,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,AdmPlan,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
Id,,,,,,,,,,,,,,,,,,,,,
1369461,219949,AD368769,7,Ingreso de Urgencias,2020-01-23 10:22:25.850,NaN,NaN,GQUINTANA,3800965,20190409,...,False,NaN,NaN,NaN,3.0,NaN,2020-01-23 09:34:00,NaN,False,NaN


In [19]:
def caseToBackground(case, backgroundByCase):
    p = backgroundByCase[case]
    if len(p) != 0:  
        text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    text = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    return text

In [20]:
import unicodedata
def remove_diacritics(text):
    """
    Returns a string with all diacritics (aka non-spacing marks) removed.
    For example "Héllô" will become "Hello".
    Useful for comparing strings in an accent-insensitive fashion.
    """
    normalized = unicodedata.normalize("NFKD", text)
    return "".join(c for c in normalized if unicodedata.category(c) != "Mn")


case, _, _ = list(interventionToCase.values())[423]
print(case)
p = backgroundByCase[case]
if len(p) != 0:  
    text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    textClean = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    print(textClean)
else:
    print('Error')

AD337645


C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.


- PERSONALES:
  7-&gt; MENARQUIA: 9ANOS 


VIDA MARITAL: 17 OS
VIDA OBSTETRICA: 18NOS
G3P2 
FUM: NO CONFIABLE 
PLANIFICACION: NIEGA 
GRUPO SANGUINEO: O+ 
CONTROLES PRENATALES: 0 

PARACLINICOS NO TRAE
ECOGRAFIA: NO TRAE
- FAMILIARES:
  NIEGA
- ALERGICOS:
  
- QUIRURGICOS:


# Background

Check from the yes / no form. 

The ones that have, try to get the most common. The rest note for manual

In [81]:
prettyPrintXML(registros[(registros.Caso == case) & registros.Asunto.str.contains('Ingreso de Urgencias')].iloc[0].RegistroXML)

<?xml version="1.0" ?>
<C Asunto="Ingreso de Urgencias" Caso="AD368769" CentroA="01" CodigoDiagnosticoRelacionado1="O623" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="Z349" IdPaciente="219949" Modulo="03" Padre="NULL" PlanAdm="20190409" Prestador="3800965" Raiz="NULL" Registro="07">
	<row NombreCampo="TipoConsulta" NombreTabla="CamposTexto" ValorCampo="OBSTETRICA"/>
	<row NombreCampo="PlanAdministradora" NombreTabla="CamposTexto" ValorCampo="DEPARTAMENTO ADMINISTRATIVO DISTRITAL DE SALUD (DADIS) / DADIS CARTAGENA EXTRANJERO"/>
	<row NombreCampo="MotivoConsulta" NombreTabla="CamposTexto" ValorCampo="&amp;quot;TENGO DOLORES&amp;quot;"/>
	<row NombreCampo="EnfermedadActual" NombreTabla="CamposTexto" ValorCampo="PACIENTE DE 23 A&amp;#209;OS QUIEN INGRESA A URGENCIA POR PRESENTAR DOLORES ABDOMINALES TIPO COLICO ASOCIADO A CONTRACCIONES. NOIEGA SALIDA DE LIQUIDO Y SANGRE POR GENITALES. REFIERE SENTIR MOVIMIENTOS FETALES. NIEGA OTRA SINTOMATOLOGIA. "/>
	<row 

# Paraclinics and background familiar

- From the antecedentes? There are many without anything in the venezuelans. Are hey applied during labour?
- Much variability -> need to clean manually, but it might be too costly

In [62]:
# Maybe I should start tokenizing
cleanWhites = re.compile("[^\S\n]+")
cleanWhites.sub(' ', text)

def cleanString(text, removeChars = '-:,;', removeWords = []):
    for c in removeChars:
        text = re.sub('(?<![0-9])\%s' %c, ' ',  text)
        text = re.sub('\%s(?![0-9])' %c, ' ',  text)

    text = cleanWhites.sub(' ', text)
    for w in removeWords:
        text = text.replace(' ' + w + ' ', ' ')
    return text.strip()

In [63]:
#Parse paraclinics
#VIH
#VDRL -
# PRUEBA RAPIDA DE SIFILIS
dict = {}
for (case, _, _) in interventionToCase.values():
    p = backgroundByCase[case]
    if len(p) == 0:
        continue
    text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    if len(r) == 0:
        continue
       # text = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    dict[case] = {'VIH': 'VIH' in text, 'VDRL' : 'VDRL' in text, 'PARACLINICOS' :  'PARACLINICOS' in text, 'SIF' : 'PRUEBA RAPIDA DE SIFILIS' in text}

In [64]:
paraclinics = pandas.DataFrame.from_dict(dict, orient = 'index')
paraclinics.sum(), len(paraclinics)

(VIH             1939
 VDRL            2077
 PARACLINICOS    2222
 SIF               93
 dtype: int64,
 2460)

In [72]:
paraclinics

,VIH,VDRL,PARACLINICOS,SIF
AD152124,False,False,True,False
AD169414,False,False,True,False
AD182404,True,True,True,False
AD191656,True,True,True,False
AD184378,True,True,True,False
...,...,...,...,...
AD368707,True,True,True,False
AD368548,False,False,True,True
AD368729,True,True,True,False
AD368822,True,True,True,False


In [ ]:
casos

In [73]:
casos.loc['AD368548']
pacientes.loc[219850]

FechaHora                   2020-01-21 18:58:00.000
Reingreso                                     False
Paciente                                     219850
Acompañante                                     NaN
DireccionAcompanante                            NaN
                                     ...           
FechaPrimeraEvolucion                           NaN
FechaUltimaEvolucion                            NaN
UFUltimaEvolucion                               NaN
PrestadorUltimaEvolucion                        NaN
UsuarioCierre                                   NaN
Name: AD368548, Length: 77, dtype: object

In [65]:
def parseAntecedentes(t):
    t = t.upper()
    antecedentes = []
    negative = ['NO', 'NIEG', 'SIN DATOS', 'NEGATIVO', 'NO REFIERE', 'SIN']
    if 'HIPERTEN' in t or 'HTA' in t:
        antecedentes.append('HTA')
    if any([n in t for n in negative]):
        antecedentes.append('None')
    if 'DIAB' in t or 'DM' in t:
        antecedentes.append('Diabetes')
    if 'ASMA' in t:
        antecedentes.append('Asma')
    if 'CARDIO' in t:
        antecedentes.append('Cardo')
    if 'PREECLAMPSIA' in t:
        antecedentes.append('Preclampsia')

    return antecedentes

In [68]:
j  = 0
noFam, noPara = 0, 0
fam = {}
para = {}
for k, r in paraclinics.iterrows(): 
    text = caseToBackground(k, backgroundByCase)
    paraclinicos = re.findall('PARACLINICOS(.*)FAMILIARES', text, re.DOTALL)
    familiares = re.findall('FAMILIARES(.*)ALERGICOS', text, re.DOTALL)
    if not paraclinicos:
        noPara += 1
    else: 
        
    else:
        para[k] = cleanString(paraclinicos[0])
    if not familiares:
        noFam += 1
    else:
        print(cleanString(familiares[0]))
        fam[k] = parseAntecedentes(cleanString(familiares[0]))
print(noFam, noPara, len(paraclinics))

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  if __name__ == '__main__':


NIEGA ANTECEDENTES DE IMPORTANCIAS
PADRE DIABETES
HIPERTENSION (PADRE)
NIEGA
NIEGA
SIN DATOS
MADRE HTA
NIEGA
MADRE HIPERTENSA.
NIEGA
NIEGA ANTECEDENTES
NIEGA
NIEGA
NEGATIVO
MADRE HTA DM TIPO 2
NIEGA
NIEGA
NIEGA
NIEGA
NIEGA
NIEGA
NIEGA
NIEGA
NIEGA
NIEGA
HTA MADRE
PADRE DIABETES
MADRE HTA 
HERMANA EPILEPSIA
NIEGA
NIEGA
HERMANO ASMA
NIEGA
NIEGA
HTA ABUELA MATERNA 
 ALERGIAS NIEGA 
 TRANSFUSIONES NO 
 QUIRURGICOS NO 
 
PARACLINICOS 
ECOGRAFIA 26-07-17 EMBARAZO DE 29.3 SEMANAS PARA HOY 39.6 SEMANAS 
ECOGRAFIA 12-09-17 EMBARAZO DE 37.1 SEMANAS PARA HOY 40.5 SEMANAS 
 
HEMOGRAMA 12-09-17 HEMOGLOBINA 11.7 HEMATOCRITO 35.3% LEUCOCITOS 9.400 PLAQUETAS 245000 
PARCIAL DE ORINA NORMAL 
 FAMILIARES 
 HTA ABUELA MATERNA
DM TIA PATERNA MADRE ASMA
MADRE DIABETICA
NIEGA
SIN DATOS
NIEGA
MADRE CANCER DE MAMA HTA
NIEGA
NIEGA
NIEGA
NIEGA
ABUELA MATERNA HTA
PADRE Y HERMANO ASMA
ABUELA MATERNA DIABETICA ABUELA PATERNA HTA
NIEGA
NIEGA
MAMA HTA
NIEGA
NIEGA
ABUELA DIABETICA TIA HIPERTENSA
NIEGA
NIEGA
HTA MADRE 

In [67]:
for c in any.values():
    print(c)

ECOGRAFIA 03/06/2016 13 SEMANAS+ OLIGOHIDRAMNIOS SEVERO
08/JUNIO/16 HEMOGRAMA HB 10,4 HTO 31 LEU 7,700
27/JUNIO/16 ANTITOXOPLASMA ( )
NO TRAE VIH NI VDRL


 5 &gt NIEGA
12 DE JULIO DEL 2016 HEMOGRAMA HB 10.9GR/DL HTC 34.5% PLQ 283.000 LEU 10.100

28/12/2016 
 
HEMOGRAMA HB 9.6 HTO 30.3 PLA 312000 
PARCIAL DE ORINA NORMAL DEGLICEMIA 63 MG/DL 
TOXOPLASMA IGG NEGATIVO IGM NEGATIVO 
RUBEOLA IGG IGM 
VIH NEGATIVO DE 12/08/2016 
VDRL NEGATIVO DE 12/08/16 
HEPATITIS B NEGATIVO 12/08/16
9/11/2016 
VDRL NO REACTIVO 
HB 10.3 HTO 31 
ANTECEDENTE ASMA
.

*** 
 
 
* 2 &gt 12/12/2012 CESAREA
12/05/2017 
VIH NEGATIVO 
VDRL NO REACTIVO 
 
HEMOGRAMA HB 11 MG/DL PLAQUETAS 225000 LEUCOCITOS 8400 
 
ANTIGENO DE SUPERFICIE PARA HEPATISIS B NEGATIVO 
UROCULTIVO NEGATIVO 
RUBEOLA IGM 2.9 
TEST O SULLIVAN 78-100
.

DE CONTROL PRENATAL
HB 10.5 HTO 33.6 LEUCO 7.900 NEUTRO 64.1 
PLAQUETAS. 189.000 
FTA NEGATIVO 
HEPATITIS B NEGATIVO 
TOXOPLASMA 
IgM Y IgG NEGATIVO 
VIH 1-2 NEGATIVO 
UROANALISIS 
LEU 1X2 CAMPO 
H

# Echographies:

- There is a lot of variability that makes it difficult to get all of them.
- In most cases I should be able to get at least one.
- Do-able: GA. If there is something more, do it manually

In [35]:
noEchos = ['no trae ecografia', 'ni ecografia']

In [38]:

embarazo = ['embarazo de', 'emb de', 'embarazo', 'emb', 'QUE REPORTA EMBARAZO DE']
query = 'Ecografi[a]?[s]?' + sep +  '(' + sep +  date + sep + '(?:%s)' % '|'.join(embarazo) + blanks + '(%s)' % floatParse  + blanks + semanas \
+ blanks  + '(:?'+ paraHoy + floatParse +  ')?' + ')+'
re.findall(query, text, re.IGNORECASE)

[]

In [40]:
blanks = '\s*'
paraHoyPossibilities = ['Para hoy', 'hoy', 'a hoy', 'para el dia de hoy']
paraHoy = '(?:%s)' % '|'.join(paraHoyPossibilities)
semanas = '(?:%s)' % '|'.join(['semanas', 'sem', 'sems'])

echoLine =  sep +  date +  sep + '(?:%s)' % '|'.join(embarazo) + blanks + '(' + '(%s)' % floatParse  + blanks + semanas  + '[,]?'\
+ blanks + paraHoy + blanks + floatParse + blanks + '(:?%s)?' % semanas + ')?'

queryEchos = '(Eco[g]?[r]?[a]?[f]?[i]?[a]?[s]?' + sep +  '(' + echoLine  + ')+)'
def searchEchos(text):
    for t in re.findall(query, text, re.IGNORECASE):
        print(t)
    #TODO: afterwards parse all echographies from the result

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [96]:
backgroundByCase['AD301211'] = getBackgroundRegister(case, registersByPatient.get_group(pId))

In [97]:
registros[registros.NumeroHistoria == 216835]

,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,AdmPlan,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
Id,,,,,,,,,,,,,,,,,,,,,
1361304,216835,AD366773,1,OBSTETRICA,2020-01-07 08:55:48.203,NaN,NaN,RFLOREZ,72133854,20190409,...,False,NaN,NaN,NaN,2.0,NaN,NaT,NaN,False,NaN
1365508,216835,AD367822,20,Registro Evaluaci&#243;n Preanastesica,2020-01-15 12:40:02.610,NaN,NaN,GDELRIO,73084739,20190409,...,False,NaN,NaN,NaN,2.0,NaN,NaT,NaN,False,NaN
1366158,216835,AD367925,94,Notas de Ingreso a Piso,2020-01-16 15:24:04.307,NaN,NaN,NCARMONA,45470969,20190409,...,False,NaN,NaN,NaN,1.0,NaN,2020-01-16 15:20:00,NaN,False,NaN
1366163,216835,AD367925,165,Registro del recién nacido,2020-01-16 15:33:44.960,NaN,Padre,sa,1047467767,20190409,...,False,NaN,NaN,NaN,3.0,NaN,NaT,NaN,False,NaN
1366194,216835,AD367925,145,NaN,2020-01-16 16:59:59.857,NaN,NaN,RDIAZ,18938424,20190409,...,False,NaN,NaN,NaN,3.0,NaN,NaT,NaN,False,NaN
1366262,216835,AD367925,94,Notas de Ingreso a Piso,2020-01-16 21:52:40.170,NaN,NaN,OSALASG,72490114,20190409,...,False,NaN,NaN,NaN,1.0,NaN,2020-01-16 21:46:00,NaN,False,NaN
1366420,216835,AD367925,106,Evoluci&#243;nMA&#209;ANA,2020-01-17 09:56:41.100,94.0,1366262,JRAMIREZ,9068204,20190409,...,True,NaN,NaN,NaN,1.0,NaN,2020-01-17 09:49:00,NaN,False,NaN
1366423,216835,AD367925,46,Registro de Incapacidad,2020-01-17 10:00:22.400,NaN,NaN,JRAMIREZ,9068204,20190409,...,False,NaN,NaN,NaN,1.0,NaN,NaT,NaN,False,NaN
1366425,216835,AD367925,450,Epicrisis de la Historia Notas de Ingreso a Piso,2020-01-17 10:01:49.257,1.0,AD367925,JRAMIREZ,9068204,20190409,...,False,NaN,NaN,NaN,1.0,NaN,NaT,NaN,False,NaN


In [ ]:
#TODO: check that operation and diagnosis are compatible.

In [ ]:
getprocID(i.XmlDescripcion)
for k,c in interventionToCase[k].iterrows():
    prettyPrintXML(c.RegistroXML)

# Patient information

In [ ]:
def getDataFromPatient(p):
    """
    Get the patient data
    """
    r = {}
    r['id'] = r.Identificacion
    r['marital'] = p.EstadoCivil
    r['etnia'] = p.Etnia #Map from id to string
    r['estudios'] = p.Escolaridad #Map from id to string
    r['nacimiento'] = p.FechaNac #Map from id to string

# New born ifnormation

- New born anthropometrics
- Birth details (quirurgic)
- Life  /dead, and also of the mum
- Whether there was a single one, or many
- C- section

# Abortion